<a href="https://colab.research.google.com/github/Mohamed-Sherif-Mahdy/Machine-Learning-for-reuse/blob/main/generic_ds_pipeline_LGBM_ASHRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os

# Get data

Download the data if needed, and make the files available locally 

In [ ]:
'''
if not kaggle:
  !pip install kaggle
  from getpass import getpass
  import os
  user = 'ahmadelsallab'
  key = '6b7ffe97ff5bc0656e325b746b72fa31'

  if '.kaggle' not in os.listdir('/root'):
      !mkdir ~/.kaggle
  !touch /root/.kaggle/kaggle.json
  !chmod 666 /root/.kaggle/kaggle.json
  with open('/root/.kaggle/kaggle.json', 'w') as f:
      f.write('{"username":"%s","key":"%s"}' % (user, key))
  !chmod 600 /root/.kaggle/kaggle.json

  !kaggle competitions download -c ashrae-energy-prediction

  !unzip -n train.csv.zip
  !unzip -n weather_train.csv.zip
  !unzip -n building_metadata.csv.zip
  !unzip -n test.csv.zip
  !unzip -n weather_test.csv.zip    
  return '.'
  '''
def get_data_from_kaggle(data_info):
    
    # Authenticate and authorize kaggle user
    !pip install kaggle
    from getpass import getpass

    user = data_info['user_info']['user']
    key = data_info['user_info']['key']

    if '.kaggle' not in os.listdir('/root'):
        !mkdir ~/.kaggle
    !touch /root/.kaggle/kaggle.json
    !chmod 666 /root/.kaggle/kaggle.json
    with open('/root/.kaggle/kaggle.json', 'w') as f:
        f.write('{"username":"%s","key":"%s"}' % (user, key))
    !chmod 600 /root/.kaggle/kaggle.json

    # Download data
    !kaggle competitions download -c {data_info['competetion_name']}
    
    # Unzip files
    for file in data_info['zip_files']:
        !unzip -n {file}


In [ ]:
def get_data(**kwargs):
    kaggle = kwargs.get('kaggle', False)
    data_info = kwargs['data_info']
    if not kaggle:
        get_data_from_kaggle(data_info=data_info)
        data_path = '.'
    else:
        import os
        data_path = os.path.join(data_info.get('kaggle_data_path', '.'), data_info['competetion_name'])
        for dirname, _, filenames in os.walk(data_path):
          for filename in filenames:
              print(os.path.join(dirname, filename))        
                                 
    return data_path

In [ ]:

kaggle=True


data_info = {
    'source': 'kaggle',
    'user_info': 
    {
        'user': 'ahmadelsallab',
        'key' : '6b7ffe97ff5bc0656e325b746b72fa31',
    },

    'competetion_name': 'ashrae-energy-prediction',

    'zip_files': ['train.csv.zip', 'weather_train.csv.zip', 'test.csv.zip', 'weather_test.csv.zip', 'building_metadata.csv.zip'],
    
    'kaggle_data_path': '/kaggle/input',
    
}


data_path = get_data(kaggle=kaggle, data_info=data_info)


/kaggle/input/ashrae-energy-prediction/test.csv
/kaggle/input/ashrae-energy-prediction/building_metadata.csv
/kaggle/input/ashrae-energy-prediction/train.csv
/kaggle/input/ashrae-energy-prediction/weather_test.csv
/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/input/ashrae-energy-prediction/weather_train.csv


# Load data
Load data from files into dataframe


In [ ]:

def load_data(**kwargs)->pd.DataFrame:
    path = kwargs['path']
    building_path = kwargs['building_path']
    weather_path = kwargs['weather_path']

    
    return pd.merge(pd.merge(pd.read_csv(path), pd.read_csv(building_path), on='building_id', how='left'), pd.read_csv(weather_path), on=['timestamp','site_id'], how='left')

In [ ]:
'''
kwargs = {}
kwargs['path'] = data_path + '/train.csv'
kwargs['building_path'] = data_path + '/building_metadata.csv'
kwargs['weather_path'] = data_path + '/weather_train.csv'
load_data(kwargs)
'''
train_path = os.path.join(data_path, 'train.csv')
building_path = os.path.join(data_path,'building_metadata.csv')
weather_train_path = os.path.join(data_path,'weather_train.csv')
#train_df = load_data(path=train_path, building_path=building_path, weather_path=weather_train_path)
#train_df.head()

# Clean data

Clean the data:
- Missing values imputation
- Outliers removal
- Memory reduction
....

In [ ]:
'''Function to reduce the DF size'''
# source: https://www.kaggle.com/kernels/scriptcontent/3684066/download

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
def clean_data(df:pd.DataFrame) -> pd.DataFrame:
    df = reduce_mem_usage(df)
    return df

In [ ]:
#train_df = clean_data(train_df)

# Features extraction

Features engineering, derived features, features selection,..etc

In [ ]:
# Select features
#features = ['meter','site_id','building_id','primary_use','square_feet','year_built','floor_count','air_temperature','dew_temperature','cloud_coverage','wind_direction','wind_speed','precip_depth_1_hr','hour','weekday', 'day', 'month', 'season']
numerical_features = ['square_feet', 'air_temperature', 'dew_temperature', 'precip_depth_1_hr'] 
cat_features = ['meter', 'site_id', 'building_id']
skewed_features = ['square_feet']
nominal_features = ['primary_use']
target_var = 'meter_reading'

In [ ]:
def extract_features(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    features = kwargs.get('features', None)
    # Derive new features
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["weekday"] = df["timestamp"].dt.weekday_name 
    df["month"] = df["timestamp"].dt.month
    df['year'] = df['timestamp'].dt.year   
    month_to_season = lambda month:(month%12 + 3)//3
    df['season'] = df['month'].apply(month_to_season)
            
    # Select only the needed features
    if features:
        df = df[features]
    
    return df

In [ ]:
#train_df = extract_features(train_df, features=numerical_features+cat_features+nominal_features+[target_var])
#train_df.head()

# Encode features

Transform and digitize/binarize features:
- Nominal to numerical features (LabelEncoder)
- OneHotEncoding (OHE) (not needed for Enitity Embedding)
- Normalization if needed (Not needed for DecisionTrees)(StandardScaler or MinMaxScaler) 
....

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
def encode_features(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    
    numerical_features = kwargs.get('numerical_features', None)
    cat_features = kwargs.get('cat_features', None)
    nominal_features = kwargs.get('nominal_features', None) 
    skewed_features = kwargs.get('skewed_features', None)
    
    # Handle skewed features
    if skewed_features:
        df[skewed_features] =  np.log1p(df[skewed_features])
    
    # Normalized numerical_features
    if numerical_features:
        df[numerical_features] = pd.DataFrame(MinMaxScaler().fit_transform(df[numerical_features].values), index=df.index, columns=numerical_features)
    
    # OHE cat features (not for DT)
    #df[cat_features] = pd.DataFrame(OneHotEncoder().fit_transform(df[cat_features].values), index=df.index, columns=cat_features)
    # Encode Categorical Data
    if nominal_features:
        le = LabelEncoder()    
        df[nominal_features] = pd.DataFrame(le.fit_transform(df[nominal_features].values), index=df.index, columns=nominal_features)

    
    return df

In [ ]:
#train_df = encode_features(train_df, numerical_features, cat_features, nominal_features, skewed_features)
#train_df.head()

# Data preprocessing pipeline


In [ ]:
# FIXME: sklearn pipeline
def data_pipeline(**kwargs):
    path = kwargs.get('path', '.')
    building_path = kwargs['building_path']
    weather_path = kwargs['weather_path']
    numerical_features = kwargs.get('numerical_features', None)
    cat_features = kwargs.get('cat_features', None)
    nominal_features = kwargs.get('nominal_features', None)
    skewed_features = kwargs.get('skewed_features', None)
    target_var = kwargs.get('target_var', None)
    size = kwargs.get('size', None)
    
    if size != None:
        df = load_data(path=path, building_path=building_path, weather_path=weather_path)[:size]
    else:
        df = load_data(path=path, building_path=building_path, weather_path=weather_path)
    df = clean_data(df)
    if target_var:
        features = numerical_features+cat_features+nominal_features+[target_var]
    else:
        features = numerical_features+cat_features+nominal_features
    df = extract_features(df, features=features)
    df = encode_features(df, 
                         numerical_features=numerical_features, 
                         cat_features=cat_features, 
                         nominal_features=nominal_features, 
                         skewed_features=skewed_features)
    
    return df
    #return encode_features(extract_features(clean_data(load_data(path=path))))

In [ ]:
size = 1000
train_df = data_pipeline(path=train_path,
                         building_path=building_path,
                         weather_path=weather_train_path,
                         numerical_features=numerical_features, 
                         cat_features=cat_features, 
                         nominal_features=nominal_features, 
                         skewed_features=skewed_features,
                         target_var=target_var,
                         size=size)

Memory usage of dataframe is 0.13 MB
Memory usage after optimization is: 0.04 MB
Decreased by 67.0%


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:355: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
train_df.head()

,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,meter,site_id,building_id,primary_use,meter_reading
0,0.407863,1.0,1.0,NaN,0,0,0,0,0.0
1,0.282316,1.0,1.0,NaN,0,0,1,0,0.0
2,0.367410,1.0,1.0,NaN,0,0,2,0,0.0
3,0.552652,1.0,1.0,NaN,0,0,3,0,0.0
4,0.751783,1.0,1.0,NaN,0,0,4,0,0.0


# Model
## Build

In [ ]:
import lightgbm as lgb
def build_model():## (arch)->Model
    
    return lgb

In [ ]:
model = build_model()

# Fit

In [ ]:
def fit(model, **kwargs):#x_train, y_train, x_val=None, y_val=None):
    x_train = kwargs['x_train']
    y_train = kwargs['y_train']
    x_val = kwargs['x_val']
    y_val = kwargs['y_val']
    params = kwargs['params']
    categorical_features = kwargs['categorical_features']
    
    d_training = model.Dataset(x_train, label=y_train, categorical_feature=categorical_features, free_raw_data=False)
    d_test = model.Dataset(x_val, label=y_val, categorical_feature=categorical_features, free_raw_data=False)

    model = model.train(params, train_set=d_training, num_boost_round=1000, valid_sets=[d_training,d_test], verbose_eval=25, early_stopping_rounds=50)
    
    return model
    

## Predict

In [ ]:
def predict(model, data):
    preds = np.expm1(model.predict(data, num_iteration=model.best_iteration))
    return preds
    

# Evaluate

In [ ]:
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse

def evaluate(gt, preds):
    score = np.sqrt(mse(gt, preds))# we use mse not msle since we already make y_val as np.log1p and so the y_pred
    return score

In [ ]:
def get_preds(models, data):
    preds = []
    for model in models:
        if  preds == []:
            preds = predict(model, data)
        else:
            preds += predict(model, data)
    preds /= len(models) 
    return preds

# Train
Fit the model to data

Inside, you may:
- Cross validation
- Split train test

Then you run the train loop, based on the training protocol

The output is a trained model

In [ ]:
from sklearn.model_selection import KFold
'''
def train(model, df:pd.DataFrame, target_var:str):#-> Model

    # Handle skew in target
    df[target_var] = df[target_var].apply(np.log1p)
    
    features = df.drop(target_var, axis = 1)
    target = df[target_var]
    
    kf = KFold(n_splits=3)
    models = []
    for train_index,test_index in kf.split(features):
        train_features = features.loc[train_index]
        train_target = target.loc[train_index]

        test_features = features.loc[test_index]
        test_target = target.loc[test_index]

        model = fit(model, train_features, train_target, test_features, test_target)
        models.append(model)
        del train_features, train_target, test_features, test_target
        gc.collect()
    return models
'''


'\ndef train(model, df:pd.DataFrame, target_var:str):#-> Model\n\n    # Handle skew in target\n    df[target_var] = df[target_var].apply(np.log1p)\n    \n    features = df.drop(target_var, axis = 1)\n    target = df[target_var]\n    \n    kf = KFold(n_splits=3)\n    models = []\n    for train_index,test_index in kf.split(features):\n        train_features = features.loc[train_index]\n        train_target = target.loc[train_index]\n\n        test_features = features.loc[test_index]\n        test_target = target.loc[test_index]\n\n        model = fit(model, train_features, train_target, test_features, test_target)\n        models.append(model)\n        del train_features, train_target, test_features, test_target\n        gc.collect()\n    return models\n'

In [ ]:
from sklearn.model_selection import KFold, train_test_split
def train(model, data:pd.DataFrame, **kwargs):#-> Model
    categorical_features = kwargs['categorical_features']
    target_var = kwargs['target_var']
    # Handle skew in target
    data[target_var] = data[target_var].apply(np.log1p)
    
    features = data.drop(target_var, axis = 1)
    target = data[target_var]
    x_train, x_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

    
    params = {
        "objective": "regression",
        "boosting": "gbdt",
        "num_leaves": 1280,
        "learning_rate": 0.05,
        "feature_fraction": 0.85,
        "reg_lambda": 2,
        "metric": "rmse",
    }
    
    model = fit(model, x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val, params=params, categorical_features=categorical_features)
    
    #print('Score on validation data: ', evaluate(y_val, get_preds([model], x_val)))
    return model



In [ ]:
categorical_features = cat_features + nominal_features # nominal are not categorical after encoding
model = train(model, train_df, target_var=target_var, categorical_features=categorical_features)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[25]	training's rmse: 1.26445	valid_1's rmse: 1.46751
[50]	training's rmse: 1.10381	valid_1's rmse: 1.42168
[75]	training's rmse: 1.03756	valid_1's rmse: 1.42064
[100]	training's rmse: 0.985275	valid_1's rmse: 1.43563
[125]	training's rmse: 0.948226	valid_1's rmse: 1.43903
Early stopping, best iteration is:
[75]	training's rmse: 1.03756	valid_1's rmse: 1.42064


In [ ]:
x_val = train_df[:100].drop(target_var, axis = 1)
y_val = train_df[:100][target_var]

print('MSE on validation data: ', evaluate(y_val, get_preds([model], x_val)))


MSE on validation data:  0.6335885952796304


In [ ]:
'''
for model in models:
    lgb.plot_importance(model)
    plt.show()
'''

NameError: name 'models' is not defined

# Test

In [ ]:
test_path = os.path.join(data_path, 'test.csv')
building_path = os.path.join(data_path,'building_metadata.csv')
weather_test_path = os.path.join(data_path,'weather_test.csv')
test_df = data_pipeline(path=test_path,
                         building_path=building_path,
                         weather_path=weather_test_path,
                         numerical_features=numerical_features, 
                         cat_features=cat_features, 
                         nominal_features=nominal_features, 
                         skewed_features=skewed_features,
                         target_var=None,
                         size=size)

Memory usage of dataframe is 0.13 MB
Memory usage after optimization is: 0.04 MB
Decreased by 67.8%


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
test_df.head()

,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,meter,site_id,building_id,primary_use
0,0.438338,1.0,0.0,NaN,0,0,0,0
1,0.303410,1.0,0.0,NaN,0,0,1,0
2,0.394863,1.0,0.0,NaN,0,0,2,0
3,0.593945,1.0,0.0,NaN,0,0,3,0
4,0.807955,1.0,0.0,NaN,0,0,4,0


In [ ]:
#gt = test_df[target]

results = get_preds([model], test_df) 


# Submission (on kaggle)

In [ ]:
def submit(preds, target, **kwargs):
    
    path = kwargs['path']
    sample_submission_file = kwargs['sample_submission_file']
    submission_file = kwargs['submission_file']
    kaggle=kwargs['kaggle']
    
    if kaggle:
      sample = pd.read_csv(path+'/'+sample_submission_file)
    else:
      !unzip -n {sample_submission_file}.zip
      sample = pd.read_csv(sample_submission_file)
    #sample[target][:1000] = np.clip(preds, 0, a_max=None)
    sample.loc[:len(preds)-1, target] = np.clip(preds, 0, a_max=None)
    sample.to_csv(submission_file, index=False)

    return sample
    

In [ ]:
submission = submit(results, target_var, path=data_path, sample_submission_file='sample_submission.csv', submission_file='submission.csv', kaggle=kaggle)
submission.head()

,row_id,meter_reading
0,0,0.664729
1,1,0.452974
2,2,0.567945
3,3,1.174288
4,4,2.302222
